In [10]:
#pip install torch torchvision pandas numpy matplotlib scikit-learn openpyxl

In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize((232, 465)),  # Resize to match your image size
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the dataset
dataset = datasets.ImageFolder(root='D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets', transform=transform)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
import pandas as pd

# Load the Excel file
tabular_data = pd.read_excel('D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\oasis_cross-sectional.xlsx')

# Preprocess the tabular data (e.g., handle missing values, encode categorical variables)
tabular_data['M/F'] = tabular_data['M/F'].map({'M': 0, 'F': 1})  # Encode gender
#tabular_data = tabular_data.fillna(tabular_data.mean())  # Fill missing values with mean

In [13]:
from torch.utils.data import Dataset

class CombinedDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.data = pd.read_excel(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the ID from the DataFrame
        img_id = self.data.iloc[idx, 0]  # Assuming ID is the first column

        # Construct the image paths
        img_files = [f for f in os.listdir(self.img_dir) if img_id in f]
        if not img_files:
            raise FileNotFoundError(f"No images found for ID: {img_id}")

        # Load and transform images
        images = []
        for img_file in img_files:
            img_path = os.path.join(self.img_dir, img_file)
            img = Image.open(img_path).convert("RGB")  # Ensure 3 channels
            if self.transform:
                img = self.transform(img)
            images.append(img)
        images = torch.stack(images)  # Stack images into a single tensor

        # Extract tabular data (excluding the ID and label columns)
        tabular_data = self.data.iloc[idx, 1:-1].values  # Adjust indices as needed
        tabular_data = torch.tensor(tabular_data, dtype=torch.float32)

        # Extract label (assuming the last column is the label)
        label = self.data.iloc[idx, -1]
        label = torch.tensor(label, dtype=torch.long)

        return images, tabular_data, label

# Create the combined dataset
combined_dataset = CombinedDataset(dataset, tabular_data)
combined_dataloader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

ValueError: Invalid file path or buffer object type: <class 'pandas.core.frame.DataFrame'>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.fc1 = nn.Linear(64 * 58 * 116, 128)  # Adjust based on your image size after pooling
        self.fc2 = nn.Linear(128 + 10, 64)  # 10 is the number of tabular features
        self.fc3 = nn.Linear(64, 4)  # 4 classes

    def forward(self, image, tabular):
        image_features = self.cnn(image)
        image_features = F.relu(self.fc1(image_features))
        combined = torch.cat((image_features, tabular), dim=1)
        combined = F.relu(self.fc2(combined))
        output = self.fc3(combined)
        return output

model = CombinedModel().to('cuda')

In [ ]:
# Training Loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, tabular_data, labels in dataloader:
        try:
            images, tabular_data, labels = images.to(device), tabular_data.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, tabular_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        except Exception as e:
            print(f"Error during training: {e}")
            continue

    return running_loss / len(dataloader)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train(model, combined_dataloader, criterion, optimizer, device)

In [ ]:
import matplotlib.pyplot as plt

# Example: Plot the loss over epochs
#plt.plot(loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Make predictions
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for images, tabular, labels in combined_dataloader:
        images, tabular, labels = images.to('cuda'), tabular.to('cuda'), labels.to('cuda')
        outputs = model(images, tabular)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()